# **Semantic search and enriched embeddings**

Popular embedding applications

* Semantic search
* Recommendation systems
* Classification tasks

## **Semantic search**

Recall that semantic search engines use embeddings to return the most semantically similar results to a search query. For example, a news website could enable semantic search by embedding news article information like the headline and topic. A user searching "computer" could then be returned a selection of computer-related headlines.

<img src= './images/semantic-search-for-online-news-website.png' width=50% height=50%>

There are three steps to semantic search:

1. __Embed__ the search query and the texts to compare against
2. Compute the __cosine distances__ between the embedded search query and other  embedded texts
3. __Extract__ the texts with the smallest cosine distances.

### **Enriched embeddings**

Here's the headlines data we'll be working with

```python
    articles = [
        {"headline": "Economic Growth Continues Amid Global Uncertainty",
         "topic": "Business",
         "keywords": ["economy","business","finance"]},
        ...
        {"headline": "1.5 Billion Tune-in to the World Cup Final",
         "topic": "Sport",
         "keywords": ["soccer","world cup","tv"]}
    ]
```

We'll embed the headline, topic, and keywords for each article. To do thus, we'll combine the information from each article into a single string that reflects the information stored in the dictionary, and the keywords are delimited by comma and space.

### **Combining features with f-string**

To combine these featurees for each article, we'll define a fucntion. This function uses `f-string`, or _formatted string_ to return the desired string structure.

```python
    articles = [..., {"headline": "1.5 Billion Tune-in to the World Cup Final",
                      "topic": "Sport",
                      "keywords": ["soccer","world cup","tv"]}]

    def create_article_text(article):
        return f"""Headline: {article['headline']}.
        Topic: {article['topic']}.
        Keywords: {', '.join(article['keywords'])}"""

    print(create_article_text(articles[-1]))
```

Output: <br>
        `Headline: 1.5 Billion Tune-in to the World Cup Final.` <br>
        `Topic: Sport. ` <br>
        `Keywords: soccer, world cup, tv`

### **Creating enriched embeddings**

To apply the function and combine the features for each article, we use a list comprehension, calling our function on each article in articles. Finally, to embed these strings, we call the `create_embeddings` function on the result. The function `create_embeddings` is defined in the notebook [`1 - What are Embeddings`](./1%20-%20What%20are%20Embeddings.ipynb). Let's recall the function, note that this creates a list of embeddings for each input using the OpenAI API.

```python
    def create_embeddings(text):
        response = client.embeddings/create(
            model="text-embedding-3-small",
            input=text
        )
    response_dict = response.model_dump(

    return [data['embedding'] for data in response_dict['data']]
    )
```

```python
    articles_text = [create_article_text(article) for article in articles]
    articles_embeddings = create_embeddings(articles_text)

    print(articles_embeddings)
```

Output: <br>
        `[[-0.019609929993748665, -0.03331860154867172, ...],` <br>
        `...` <br>
        `[..., -0.014373429119586945, -0.005235843360424042]]`

### **Computing distances**

Now we can compute cosine distances. 

We'll define a function called `find_n_closest`, that takes a `query_vector` (the embedded search query), and `embeddings` to compare against (our embedded articles), and returns the `n` most similar results based on their cosine distances. 

For each embedding, we calculate the cosine distance to the `query_vector`, and store it in a dictionary along with the embedding's index, which we append to a list called `distances`. 

To sort the distances list by the `distance` key in each dictionary, we use the sorted function and its key argument. The key argument takes a function to evaluate each dictionary in distances and sort by; in this case, it's a `lambda` function that accesses the distance key from each dictionary. 

```python
    from scipy.spatial import distance

    def find_n_closest(query_vector, embeddings, n=3):
        distances = []
        for index, embedding in enumerate(embeddings):
            dist = distance.cosine(query_vector, embedding)
            distances.append({'distance': dist, 'index': index})

        distances_sorted = sorted(distances, key=lambda x: x['distance'])
        return distances_sorted[0:n]
```

We will query our embeddings using the text, "AI". 

First, we embed the search query using our `create_embeddings` function and extract its embeddings by 0-indexing. 

Next, we use the `find_n_closest` function to find the `3` closest `hits` based on our article_embeddings. 

Finally, to extract the most similar headlines, we loop through each hit, using the hit's index to subset the corresponding headline, and print. 

```python
    query_text = "AI"
    query_vector = create_embeddings(query_text)[0]

    hits = find_n_closest(query_vector, article_embeddings)

    for hit in hits:
        article = articles[hit['index']]
        print(article['headline'])
```

Output: <br>
<img src = './images/search_result.png' width = 70% height = 70%>

As we'd expect, the top result scpecifically mentions AI, and the others are on similar topics.

## **Recommendation System**

Recommendation systems work almost exactly the same as semantic search engines! We have a group of items that we potentially want to recommend, and at least one data point that we want to recommend based on. 

1. Embed the potential recommendations and the data point we have
2. Calculate cosine distances
3. Recommend the item or items that are closest in the vector space.

This similarity in approach means that a lot of the code is exactly the same.

### **Example: Recommmend articles**

We're returning to the news article dataset to design a recommendation system that recommends the three most similar articles to the one currently being read.

```python
    articles = [
        {"headline": "Economic Growth Continues Amid Global Uncertainty",
         "topic": "Business",
         "keywords": ["economy","business","finance"]},
        ...
        {"headline": "1.5 Billion Tune-in to the World Cup Final",
         "topic": "Sport",
         "keywords": ["soccer","world cup","tv"]}
    ]
```

This recommendation will be based on the article's headline, topic, and keywords, stored in a dictionary called `current_article`: 

```python
    current_article = {
        "headline": "How NVIDIA GPUs Could Decide Who Wins the AI Race",
        "topic": "Tech",
        "keywords": ["ai","business","computers"]
    }
```

To prepare both of these objects for embedding, we'll first need to combine the features into a single string for each article.

We'll use `create_article_text` function. Let's recall the function:

```python
    def create_article_text(article):
        return f"""Headline: {article['headline']}.
        Topic: {article['topic']}.
        Keywords: {', '.join(article['keywords'])}"""
```

### **Combining Features**

To combine the features, we call the function on each article in articles using a list comprehension, and do the same for the current article. By printing, we can see that the function correctly formatted the current article's information.

```python
    articles_text = [create_article_text(article) for article in articles]
    current_article_text = create_article_text(current_article)
    print(current_article_text)
```
<img src= '/Users/erensen/Downloads/DataCamp/Introduction to Embeddings with the OpenAI API/images/current_article.png' width=65% height=65%>

### **Creating Embeddings**

Next, we embed both sets of article strings using the `create_embeddings` function from earlier, which allows us to create requests to the OpenAI embedding model in a more repeatable way.

```python
    def create_embeddings(text):
        response = client.embeddings/create(
            model="text-embedding-3-small",
            input=text
        )
    response_dict = response.model_dump(

    return [data['embedding'] for data in response_dict['data']]
    )
```

### **Finding the most similar article**

Finally, to compute the cosine distances and extract the nearest distances, we'll use our `find_n_closest` function, which computes the cosine distances, sorts them, and returns the n smallest and their index. We'll call this function on both sets of embedded articles, and then loop through the results, returning the headline of each article in the three most similar articles.

```python
    def find_n_closest(embedding, embeddings, n=3):
        distances=[]
        for index, embedding in enumerate(embeddings):
            dist = spatial.distance.cosine(query_vector, embedding)
            distances.append('distance': dist, 'index': index)
        distances_sorted = sorted(distances, key=lambda x: x['distance'])
        return distances_sorted[:n]

hits = find_n_closest(query_embedding, embeddings)
for hit in hits:
    article = articles[hit['index']]
    print(article['headline'])
```

output: <br>
<img src = './images/recommend-articles.png' width=65% height=65%>

This is a good start, but a more sophisticated system would use not only the current article to base the recommendations on, _but also the other articles in the user's history_.

### **Adding user history**

Let's consider that a user has visited two articles, stored in `user_history`

```python
    user_history = [
        {"headline": "How NVIDIA GPUs Could Decide Who Wins the AI Race",
         "topic": "Tech",
         "keywords": ["ai", "business", "computer"]},
        {"headline": "Tech Giant Buys 49% Stake In AI Startup",
         "topic": "Tech",
         "keywords": ["business", "AI"]}
    ]
```

How can we provide recommendations based on multiple data points?

### **Recommandation on multiple data points**

This is the situation we have, where the user has seen two articles, embedded in blue, and we want to recommend the most similar article. Articles they haven't seen yet are shown in red. 

<img src = './images/situation.png' width=25% height=25%>

To find the most similar vector to two vectors, 
1. We'll combine the two vectors into one by taking the mean. 
2. Then, we'll compute cosine distances as we did before,
3. Then
    * recommend the closest vector. <br>

    <img src = './images/take-mean.png' width=25% height=25%>

    * If the nearest point has already been viewed, we'll make sure to return the nearest unseen article.

    <img src = './images/recommend-the-unrecommended.png' width=25% height=25%>

The first two steps to embed the user_history are the same as before, combining the features for each article, and embedding the resulting strings. The only difference is that we take the mean to aggregate the two vectors into one that we can compare with the other articles. 

For the articles to recommend, we filter the list so it only contains articles _not_ in the `user_history`. 

Then, as before, we combine the features and embed the text.

```python
    def create_article_text(article):
        return f"""Headline: {article['headline']}
    Topic: {article['topic']}
    Keywords: {", ".join(article['keywords'])}"""

    history_texts = [create_article_text(article) for article in user_history]
    history_embeddings = create_embeddings(history_texts)

    mean_history_embedding = np.mean(history_embeddings, axis=0)

    article_filtered = [article for article in articles if article not in user_history]
    article_texts = [create_article_text(article) for article in article_filtered]
    article_embeddings = create_embeddings(article_texts)
```

Finally, we compute the cosine distances using the same find_n_closest function, only this time, passing it the mean of the embedded user history. Then, we subset the filtered article list to find the most similar articles. 

```python
    hits = find_n_closest(mean_history_embedding, article_embeddings)
    
    for hit in hits:
        article = articles_filtered[hit['index']]
        print(article['headline'])
```

Output: <br>
<img src= './images/most-similar-articles.png' width=60% height=60%>

Notice that the article headlined, "Tech Giant Buys 49% Stake In AI Startup" wasn't recommended, as the user had already seen it.

## Example

In this exercise, you'll make a recommendation system for an online retailer that sells a variety of products. This system recommends three similar products to users who visit a product page but don't purchase, based on the last product they visited.
You've been provided with a list of dictionaries of `products` available in `/datasets/`, and a dictionary for the last product the user visited, stored in `last_product`.

In [4]:
from openai import OpenAI
from scipy.spatial import distance      # to calculate cosine distance
import numpy as np                      # to calculate the mean of the embedded user_history
import json                             # to load the products.json file
import os                               # to get the current working directory
from dotenv import load_dotenv          # to load the .env file

# Load the .env file
load_dotenv()

# Get the API key from the .env file
api_key = os.getenv("OPENAI_API_KEY")

client = OpenAI(api_key= api_key)

with open('./datasets/products.json', 'r') as file:
    products = json.load(file)

with open('./datasets/last_product.json', 'r') as file:
    last_product = json.load(file)

# Create a function to create embeddings for product_descriptions - returns a list of embeddings for each product description
def create_embeddings(text):
    response = client.embeddings.create(
        model = "text-embedding-3-small",
        input = text
    )
    response_dict = response.model_dump()
    return [data['embedding'] for data in response_dict['data']]

# Create a list of product descriptions from the products list using list comprehension
product_descriptions = [product['short_description'] for product in products]
response = client.embeddings.create(
    model = "text-embedding-3-small",
    input = product_descriptions
)
response_dict = response.model_dump()

# Extract the embeddings from response_dict and store in products.
# Now the products list has an additional key 'embedding' for each product, placed after the 'features' key.
for i, product in enumerate(products):
    product['embedding'] = response_dict['data'][i]['embedding']

# Create two lists: one for categories and one for embeddings
categories = [product['category'] for product in products]
embeddings = [product['embedding'] for product in products]

# Define a function to combine the relevant features into a single string
def create_product_text(product):
    return f"""Title: {product['title']}
    Description: {product['short_description']}
    Category: {product['category']}
    Features: {"; ".join(product['features'])}"""

# Combine the features for each product
product_texts = [create_product_text(product) for product in products]

# Create the embeddings from product_texts
product_embeddings = create_embeddings(product_texts)


# Returns the n closest distances and their indexes between query_vector and embeddings, based on cosine distance
def find_n_closest(query_vector, embeddings, n=3):
    distances = []
    for index, embedding in enumerate(embeddings):
        dist = distance.cosine(query_vector, embedding)       # Calculate the cosine distance between the query vector and embedding
        distances.append({"distance": dist, "index": index})  # Append the distance and index to distances
    distances_sorted = sorted(distances, key= lambda x: x['distance'])      # Sort distances by the distance key

    # Return the first n elements in distances_sorted
    return distances_sorted[:n]

# Create a list of product texts using list comprehension
last_product_text = create_product_text(last_product)
product_texts = [create_product_text(product) for product in products]

# Embed last_product_text and product_texts
last_product_embeddings = create_embeddings(last_product_text)[0]   # 0-index because we are embedding a single text
product_embeddings = create_embeddings(product_texts)

# Find the three smallest cosine distances and their indexes
hits = find_n_closest(last_product_embeddings, product_embeddings)

for hit in hits:
  product = products[hit['index']]
  print(product['title'])

Robot Building Kit
LEGO Space Shuttle
Designer Makeup Brush Set


## Example

For many recommendation cases, such as film or purchase recommendation, basing the next recommendation on one data point will be insufficient. In these cases, you'll need to embed all or some of the user's history for more accurate and relevant recommendations.

In this exercise, you'll extend your product recommendation system to consider all of the products the user has previously visited, which are stored in a list of dictionaries called `user_history` under `/datasets/`.

In [ ]:
from openai import OpenAI
from scipy.spatial import distance      # to calculate cosine distance
import numpy as np                      # to calculate the mean of the embedded user_history
import json                             # to load the products.json file
import os                               # to get the current working directory
from dotenv import load_dotenv          # to load the .env file

# Load the .env file
load_dotenv()

# Get the API key from the .env file
api_key = os.getenv("OPENAI_API_KEY")

client = OpenAI(api_key= api_key)

with open('./datasets/products.json', 'r') as file:
    products = json.load(file)
    
with open('./datasets/user_history.json', 'r') as file:
    user_history = json.load(file)
    
# Create a function to create embeddings for product_descriptions - returns a list of embeddings for each product description
def create_embeddings(text):
    response = client.embeddings.create(
        model = "text-embedding-3-small",
        input = text
    )
    response_dict = response.model_dump()
    return [data['embedding'] for data in response_dict['data']]

# Create a list of product descriptions from the products list using list comprehension
product_descriptions = [product['short_description'] for product in products]
response = client.embeddings.create(
    model = "text-embedding-3-small",
    input = product_descriptions
)
response_dict = response.model_dump()

# Extract the embeddings from response_dict and store in products.
# Now the products list has an additional key 'embedding' for each product, placed after the 'features' key.
for i, product in enumerate(products):
    product['embedding'] = response_dict['data'][i]['embedding']

# Create two lists: one for categories and one for embeddings
categories = [product['category'] for product in products]
embeddings = [product['embedding'] for product in products]

# Define a function to combine the relevant features into a single string
def create_product_text(product):
    return f"""Title: {product['title']}
    Description: {product['short_description']}
    Category: {product['category']}
    Features: {"; ".join(product['features'])}"""

# Combine the features for each product
product_texts = [create_product_text(product) for product in products]

# Create the embeddings from product_texts
product_embeddings = create_embeddings(product_texts)


# Returns the n closest distances and their indexes between query_vector and embeddings, based on cosine distance
def find_n_closest(query_vector, embeddings, n=3):
    distances = []
    for index, embedding in enumerate(embeddings):
        dist = distance.cosine(query_vector, embedding)       # Calculate the cosine distance between the query vector and embedding
        distances.append({"distance": dist, "index": index})  # Append the distance and index to distances
    distances_sorted = sorted(distances, key= lambda x: x['distance'])      # Sort distances by the distance key

    # Return the first n elements in distances_sorted
    return distances_sorted[:n]

# Prepare and embed the user_history, and calculate the mean embeddings
history_texts = [create_product_text(product) for product in user_history]
history_embeddings = create_embeddings(history_texts)
mean_history_embeddings = np.mean(history_embeddings, axis=0)

# Extract the titles of products in the user's history
history_titles = [product['title'] for product in user_history]

# Filter products to remove any in user_history based on title
products_filtered = [product for product in products if product['title'] not in history_titles]

# Combine product features and embed the resulting texts
product_texts = [create_product_text(product) for product in products_filtered]
product_embeddings = create_embeddings(product_texts)

hits = find_n_closest(mean_history_embeddings, product_embeddings)

for hit in hits:
    product = products_filtered[hit['index']]
    print(product['title'])

Robot Building Kit
Interactive Robot Pet
LEGO Space Shuttle


## **Embeddings for classification tasks**

Classification tasks can take different forms, but generally, they involve assigning labels to items. Common tasks include __categorization__, such as categorizing headlines into different topics; and __sentiment analysi__s, such as classifying reviews as positive or negative. Embeddings can be used for both of these cases, utilizing the model's ability to capture semantic meaning.

We'll be using a type of a classification called __zero-shot classification__, which  means that the classifications won't be based on labeled examples.

Let's look at how that works with an example on classyfing news articles by topic.

Process:
1. Embed class descriptions
  * Here we use four classes: tech, science, sports, and business.
  * We embed these labels and use them as reference points to base the classificaton on
2. Embed the articles to classify
3. Calculate cosine distances to each embedded label
4. Assign the article the label with the smallest cosine distance.

```python
topics = [
    {'label': 'Tech'},
    {'label': 'Science'},
    {'label': 'Sports'},
    {'label': 'Business'},
]
```

We'll categorize using the label itself, so the first step is to extract the labels as a single list and use these as the class descriptions. Then embed each topic label using the `create_embeddings` custom function that makes a call to the OpenAI embedding model.

```python
class_descriptions = [topic['label'] for topic in topics]
class_embeddings = create_embeddings(class_descriptions)
```

Here's the article we want to classify:

```python
article = {"headline": "How NVIDIA GPUs Could Decide Who Wins the AI Race",
           "keywords": ["ai", "business", "computers"]
```

The first step is to combine the headline and the keywordd onformation into a single string that we can embed. We do this by defining a custom function that uses F-string to concatenate the headline and the keywords inside a nicely formatted string.

```python
def create_article_text(article):
    return f"""Headline: {article['headline']}
    Keywords: {', '.join(article['keywords'])}"""
```

Finally, we embed the text by calling `create_embeddings()` again, remembering to zero-index the list returned so we have a single list of numbers.

```python
article_text = create_article_text(article)
article_embedding = create_embeddings(article_text)[0]
```

Now that we have the embeddings, it's time for the cosine distance calculations. The following block is a modified version of the `find_n_closest()` custom function, where instead of returning n results, we only want one: the nearest label. This means that instead of sorting by distance, we can find the minimum using the min function. Calling this function will return the distance and index of this label.

```python
def find_closest(query_vector, embeddings):
    distances=[]
    for index, embedding in enumerate(embeddings):
        dist = distance.cosine(query_vector, embedding)
        distances.append({"distance": dist, "index": index})
    return min(distances, key=lambda x: x["distance"])

closest = find_closest(article_embeddings, class_embeddings)
```

Finally, we can use this index to subset the topics dictionary and extract the label. Printing the result, returns the Business label.

```python
label = topics[closest['index']['label']]
print(label)
```

Output: <br>
`Business`

But this does not seem right!. If we take another look at the article we're classifying, we can see that the headline indicates that the focus of the article is on tech; it's likely that the model captured the business keyword which resulted in the mislabeling!

The limitation in our approach that led to this was the clss descriptions lacked detail. The word "business" or "tech" does not contain much meaning on its own for the model to capture, so a better approach would be use more detailed class descriptions.

Let's try this again, but instead of using the labels as the descriptions, we use short descriptions to represent each class. 

```python
topics = [
    {'label': 'Tech', 'description': 'A news article about technology'},
    {'label': 'Science', 'description': 'A news article about science'},
    {'label': 'Sport', 'description': 'A news article about sports'},
    {'label': 'Business', 'description': 'A news article about business'},
]
```

The steps here are almost identical. We extract the descriptions in a single list, this time using the description key, and embed them. The rest of the code is the same! 

```python
class_descriptions = [topic['description'] for topic in topics]
class_embeddings = create_embeddings(class_descriptions)

[...]

label = topics [closest['index']]['label']
print(label)
```

This time, when we print the result, we cans ee that the model classified correctly.

Output: <br>
`Tech`

## **Example**

You've been provided with a small sample of restaurant reviews, stored in reviews, and sentiment labels stored in sentiments:

```python
sentiments = [{'label': 'Positive'},
              {'label': 'Neutral'},
              {'label': 'Negative'}]

reviews = ["The food was delicious!",
           "The service was a bit slow but the food was good",
           "The food was cold, really disappointing!"]
```

You'll use zero-shot classification to classify the sentiment of these reviews by embedding the reviews and class labels.

The create_embeddings() function you created previously is also available to use.

__Instruction__

* Create a list of class descriptions from the labels in the `sentiments` dictionary using a list comprehension.
* Embed `class_descriptions` and `reviews` using the `create_embeddings()` function.

In [5]:
sentiments = [{'label': 'Positive'},
              {'label': 'Neutral'},
              {'label': 'Negative'}]

reviews = ["The food was delicious!",
           "The service was a bit slow but the food was good",
           "The food was cold, really disappointing!"]

# Create a list of class descriptions from the sentiment labels
class_descriptions = [sentiment['label'] for sentiment in sentiments]

# Embed the class_descriptions and reviews
class_embeddings = create_embeddings(class_descriptions)
review_embeddings = create_embeddings(reviews)

Now that you've calculated the embeddings, it's time to compute the cosine distances and extract the most similar label.

You'll do this by defining a function called `find_closest()`, which can be used to compare the embeddings between one vector and multiple others, and return the nearest distance and its index. You'll then loop over the reviews and and use `find_closest()` to find the closest distance for each review, extracting the classified label using the index.

The `class_embeddings` and `review_embeddings` objects you created in the last exercise are available for you to use, as well as the `reviews` and `sentiments`.

__Instructions__
* Define a function called `find_closest()` that returns the distance and index of the most similar embedding to the `query_vector`.
* Use `find_closest()` to find the closest distance between each review's embeddings and the `class_embeddings`.
* Use the '`index'` of closest to subset `sentiments` and extract the `'label'`.

In [6]:
# Define a function to return the minimum distance and its index
def find_closest(query_vector, embeddings):
  distances = []
  for index, embedding in enumerate(embeddings):
    dist = distance.cosine(query_vector, embedding)
    distances.append({"distance": dist, "index": index})
  return min(distances, key=lambda x: x["distance"])

for index, review in enumerate(reviews):
  # Find the closest distance and its index using find_closest()
  closest = find_closest(review_embeddings[index], class_embeddings)
  # Subset sentiments using the index from closest
  label = sentiments[closest['index']]['label']
  print(f'"{review}" was classified as {label}')

"The food was delicious!" was classified as Positive
"The service was a bit slow but the food was good" was classified as Negative
"The food was cold, really disappointing!" was classified as Negative


One of the last predicted labels (second one) didn't seem representative of the review; this was probably down to the lack of information being captured when we're only embedding the class labels. This time, descriptions of each class will be embedded instead, so the model better "understands" that you're classifying restaurant reviews.

The following objects are available for you to use:

```python
sentiments = [{'label': 'Positive',
               'description': 'A positive restaurant review'},
              {'label': 'Neutral',
               'description':'A neutral restaurant review'},
              {'label': 'Negative',
               'description': 'A negative restaurant review'}]

reviews = ["The food was delicious!",
           "The service was a bit slow but the food was good",
           "The food was cold, really disappointing!"]
```

__Instructions__
* Extract a list containing the sentiment descriptions and embed them.

In [7]:
sentiments = [{'label': 'Positive',
               'description': 'A positive restaurant review'},
              {'label': 'Neutral',
               'description':'A neutral restaurant review'},
              {'label': 'Negative',
               'description': 'A negative restaurant review'}]

reviews = ["The food was delicious!",
           "The service was a bit slow but the food was good",
           "The food was cold, really disappointing!"]

# Extract and embed the descriptions from sentiments
class_descriptions = [sentiment['description'] for sentiment in sentiments]
class_embeddings = create_embeddings(class_descriptions)
review_embeddings = create_embeddings(reviews)

def find_closest(query_vector, embeddings):
  distances = []
  for index, embedding in enumerate(embeddings):
    dist = distance.cosine(query_vector, embedding)
    distances.append({"distance": dist, "index": index})
  return min(distances, key=lambda x: x["distance"])

for index, review in enumerate(reviews):
  closest = find_closest(review_embeddings[index], class_embeddings)
  label = sentiments[closest['index']]['label']
  print(f'"{review}" was classified as {label}')


"The food was delicious!" was classified as Positive
"The service was a bit slow but the food was good" was classified as Neutral
"The food was cold, really disappointing!" was classified as Negative
